## Imports

In [1]:
import pandas as pd

In [14]:
# Load the provided CSV file
file_path = './Dataset/Physiology.csv'
data = pd.read_csv(file_path)
data.head()

,patient_id,date,device_type,value,unit
0,0697d,2019-06-28 13:42:09,Body Temperature,36.072,Cel
1,0697d,2019-06-28 13:42:44,Body Temperature,35.831,Cel
2,0697d,2019-06-28 13:43:40,Body Temperature,35.831,Cel
3,0697d,2019-06-28 13:45:15,Systolic blood pressure,165.000,mm[Hg]
4,0697d,2019-06-28 13:45:15,Diastolic blood pressure,82.000,mm[Hg]


In [15]:
data['date'] = pd.to_datetime(data['date'])
data.dtypes

patient_id             object
date           datetime64[ns]
device_type            object
value                 float64
unit                   object
dtype: object

In [8]:
"""
IGNORE FOR NOW

# Separate the date and time into two columns
data[['Date', 'Time']] = data['date'].str.split(' ', expand=True)

# Exclude rows where device_type is 'Body Weight', 'Skin Temperature', or 'Total Body Water'
filtered_data = data[~data['device_type'].isin(['Body Weight', 'Skin Temperature', 'Total Body Water'])]
filtered_data = filtered_data.drop(['date'], axis=1)
filtered_data['DateTime'] = pd.to_datetime(filtered_data_updated['Date'] + ' ' + filtered_data_updated['Time'])

filtered_data.head()

"""

,patient_id,device_type,value,unit,Date,Time
0,0697d,Body Temperature,36.072,Cel,2019-06-28,13:42:09
1,0697d,Body Temperature,35.831,Cel,2019-06-28,13:42:44
2,0697d,Body Temperature,35.831,Cel,2019-06-28,13:43:40
3,0697d,Systolic blood pressure,165.000,mm[Hg],2019-06-28,13:45:15
4,0697d,Diastolic blood pressure,82.000,mm[Hg],2019-06-28,13:45:15


In [17]:
def assign_time_window(row):
    # Extracting hour and minute from the datetime
    hour = row.hour
    minute = row.minute

    # The first window is based on the hour itself
    start_hour_1 = hour
    end_hour_1 = (start_hour_1 + 1) % 24  # Wrapping around midnight

    # The second window depends on whether the minute is >= 30
    if minute >= 30:
        start_hour_2 = hour
    else:
        start_hour_2 = hour - 1 if hour > 0 else 23  # Wrapping around midnight

    end_hour_2 = (start_hour_2 + 1) % 24

    # Formatting the time windows
    window1 = f"{start_hour_1:02d}:00-{end_hour_1:02d}:00"
    window2 = f"{start_hour_2:02d}:30-{end_hour_2:02d}:30"

    return [window1, window2]

# Applying the function to each row and expanding the results into two columns
data[['window_1', 'window_2']] = pd.DataFrame(data['date'].apply(assign_time_window).tolist(), index=data.index)

data.head()

,patient_id,date,device_type,value,unit,window_1,window_2
0,0697d,2019-06-28 13:42:09,Body Temperature,36.072,Cel,13:00-14:00,13:30-14:30
1,0697d,2019-06-28 13:42:44,Body Temperature,35.831,Cel,13:00-14:00,13:30-14:30
2,0697d,2019-06-28 13:43:40,Body Temperature,35.831,Cel,13:00-14:00,13:30-14:30
3,0697d,2019-06-28 13:45:15,Systolic blood pressure,165.000,mm[Hg],13:00-14:00,13:30-14:30
4,0697d,2019-06-28 13:45:15,Diastolic blood pressure,82.000,mm[Hg],13:00-14:00,13:30-14:30


In [18]:
# Function to convert window and date into a datetime object with the start time of the window
def window_to_datetime(row, window_col):
    # Extracting date and start time from the row
    date = row['date'].date()
    start_time = row[window_col].split('-')[0]

    # Combining date and start time into a datetime object
    return pd.to_datetime(f"{date} {start_time}")

# Applying the function to convert window_1 and window_2
data['window_1'] = data.apply(window_to_datetime, window_col='window_1', axis=1)
data['window_2'] = data.apply(window_to_datetime, window_col='window_2', axis=1)

# Display the first 20 rows with the updated columns
data.head(20)

,patient_id,date,device_type,value,unit,window_1,window_2
0,0697d,2019-06-28 13:42:09,Body Temperature,36.072,Cel,2019-06-28 13:00:00,2019-06-28 13:30:00
1,0697d,2019-06-28 13:42:44,Body Temperature,35.831,Cel,2019-06-28 13:00:00,2019-06-28 13:30:00
2,0697d,2019-06-28 13:43:40,Body Temperature,35.831,Cel,2019-06-28 13:00:00,2019-06-28 13:30:00
3,0697d,2019-06-28 13:45:15,Systolic blood pressure,165.000,mm[Hg],2019-06-28 13:00:00,2019-06-28 13:30:00
4,0697d,2019-06-28 13:45:15,Diastolic blood pressure,82.000,mm[Hg],2019-06-28 13:00:00,2019-06-28 13:30:00
5,0697d,2019-06-28 13:45:15,Heart rate,42.000,beats/min,2019-06-28 13:00:00,2019-06-28 13:30:00
6,0697d,2019-06-28 14:25:07,Body Temperature,36.599,Cel,2019-06-28 14:00:00,2019-06-28 13:30:00
7,0697d,2019-06-28 14:25:37,Body Temperature,36.599,Cel,2019-06-28 14:00:00,2019-06-28 13:30:00
8,0697d,2019-06-28 14:40:13,Body weight,86.300,kg,2019-06-28 14:00:00,2019-06-28 14:30:00
9,0697d,2019-06-28 14:40:13,O/E - muscle mass,64.500,kg,2019-06-28 14:00:00,2019-06-28 14:30:00


In [19]:
data.dtypes

patient_id             object
date           datetime64[ns]
device_type            object
value                 float64
unit                   object
window_1       datetime64[ns]
window_2       datetime64[ns]
dtype: object

In [9]:
df = pd.DataFrame(columns=['Patient ID', 'Body Temperature', 'Systolic Blood Pressure', 'Diastolic Blood Pressure', 'Heart Rate', 'O/E - Muscle Mass', 'Agitation'])
df.head()

,Patient ID,Body Temperature,Systolic Blood Pressure,Diastolic Blood Pressure,Heart Rate,O/E - Muscle Mass,Agitation
